In [80]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
NUM_PX = 64
LAYER_DIMS = [12288, 5, 1] 

In [50]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
def delete_zone_identifier(filenames, directory):
    for filename in filenames:
        try:
            name = filename.split(":")[1]
        except:
            name = ""
        if name == "Zone.Identifier":
            os.remove(directory + "/" + filename)

In [52]:
filenames_train = os.listdir("train/train")
delete_zone_identifier(filenames_train, "train/train")
filenames_test = os.listdir("test1/test1")
delete_zone_identifier(filenames_test, "test1/test1")

In [53]:
def preprocess_image(file_path, target_size):
    image = cv2.imread(file_path)
    image = cv2.resize(image, target_size)
    image = np.array(image, dtype=np.float32)
    image = image / 255.0
    return image

In [54]:
def parse_images(filenames, directory, limit_dogs, limit_cats):
    for filename in filenames:
        category = filename.split('.')[0]
        if category == "dog":
            limit_dogs -= 1
            if limit_dogs < 0:
                os.remove(directory + "/" + filename)
        else:
            limit_cats -= 1
            if limit_cats < 0:
                os.remove(directory + "/" + filename)

In [55]:
filenames_train = os.listdir("train/train")
parse_images(filenames_train, "train/train", 500, 500)
filenames_test = os.listdir("test1/test1")
parse_images(filenames_test, "test1/test1", 100, 100)

In [56]:
def load_dataset(filenames):
    images = []
    labels = []
    for filename in filenames:
        image = preprocess_image("train/train/" + filename, (64, 64))
        images.append(image)
        category = filename.split('.')[0]
        if category == "dog":
            labels.append(1)
        else:
            labels.append(0)
    return np.array(images), np.array(labels)

In [57]:
filenames = os.listdir("train/train")
train_set_x_orig, train_set_y = load_dataset(filenames_train)

In [58]:
train_set_x = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
train_set_y = train_set_y.reshape(1, train_set_y.shape[0])

In [59]:
train_set_x.shape, train_set_y.shape

((12288, 1000), (1, 1000))

In [60]:
print(train_set_x, train_set_y)

[[0.16862746 0.827451   0.1882353  ... 0.93333334 0.22352941 0.2627451 ]
 [0.17254902 0.8509804  0.1882353  ... 0.9529412  0.32941177 0.36078432]
 [0.1882353  0.84705883 0.1882353  ... 0.9490196  0.4627451  0.5176471 ]
 ...
 [0.57254905 0.34117648 0.7254902  ... 0.9843137  0.54901963 0.28235295]
 [0.5254902  0.39607844 0.73333335 ... 1.         0.5764706  0.38039216]
 [0.49803922 0.41568628 0.73333335 ... 0.7058824  0.6509804  0.5529412 ]] [[1 0 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 1 1 0 0
  1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1
  1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1
  1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1
  1 0 0 0 1 1 0 0 1 1 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0
  0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1 1
  0 1 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0
  0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 

In [61]:
def initialize_parameters(layer_dims):
    np.random.seed(2)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters

In [62]:
parameters = initialize_parameters(LAYER_DIMS)
print(parameters)

{'W1': array([[-0.00416758, -0.00056267, -0.02136196, ..., -0.01275513,
         0.00733372,  0.01166127],
       [ 0.00016076,  0.00699494, -0.02144254, ..., -0.00569705,
        -0.00276633,  0.0011016 ],
       [-0.00553107,  0.00678748, -0.00870469, ...,  0.00174631,
         0.00395599, -0.00136441],
       ...,
       [ 0.00137381,  0.00531559, -0.00411322, ...,  0.00274936,
         0.02391327, -0.00872727],
       [-0.01886398, -0.00101521,  0.00443469, ...,  0.00105953,
         0.0073419 ,  0.00461946],
       [-0.01062171, -0.00365029, -0.00166228, ...,  0.00578868,
        -0.00498871, -0.00829043]]), 'b1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]]), 'W2': array([[ 0.00102192,  0.00040951, -0.01263179, -0.00295767,  0.00418341,
         0.00043563, -0.00196525,  0.00336217, -0.00209281, -0.00598241]]), 'b2': array([[0.]])}


In [63]:
def RELU(Z):
    cache = Z
    A = np.maximum(0, Z)
    return A, cache

In [64]:
def sigmoid(Z):
    cache = Z
    A = 1 / (1 + np.exp(-Z))
    return A, cache

In [65]:
def linear_forward(A, W, b):
    Z = np.matmul(W, A) + b
    cache = (A, W, b)
    return Z, cache

In [66]:
def linear_activation_forward(prev_A, W, b, activ_funct):
    if activ_funct == "relu":
        Z, linear_cache = linear_forward(prev_A, W, b)
        A, activation_cache = RELU(Z)

    elif activ_funct == "sigmoid":
        Z, linear_cache = linear_forward(prev_A, W, b)
        A, activation_cache = sigmoid(Z)
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [67]:
def forward_propagation(X, parameters):
    caches = []
    L = len(parameters) //2
    A = X

    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b'+str(l)], "relu")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], "sigmoid")
    caches.append(cache)

    return AL, caches

In [68]:
def cost_function(AL, Y):
    m = Y.shape[1]
    # element1 = np.multiply(Y, np.log(AL))
    # element2 = np.multiply((1 - Y), np.log(1 - AL))
    # cost = (-1 / m) * np.sum(element1 + element2)
    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    cost = np.squeeze(cost)

    return cost

In [69]:
def RELU_backward(dA, activ_cache):
    Z = activ_cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    
    return dZ

In [70]:
def sigmoid_backward(dA, activ_cache):
    Z = activ_cache

    x = 1 / (1 + np.exp(-Z))
    dZ = dA * x * (1-x)

    return dZ

In [71]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = (1 / m) * np.matmul(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.matmul(W.T, dZ)
    
    return dA_prev, dW, db

In [72]:
def linear_activation_backward(dA, cache, activ_funct):
    linear_cache, activ_cache = cache
    if activ_funct == "relu":
        dZ = RELU_backward(dA, activ_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activ_funct == "sigmoid":
        dZ = sigmoid_backward(dA, activ_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [73]:
def backward_propagation(AL, Y, caches):
    gradients = {}
    L = len(caches)
    Y = Y.reshape(AL.shape)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    dA_prev, dW, db = linear_activation_backward(dAL, current_cache, "sigmoid")
    gradients['dA'+str(L-1)] = dA_prev
    gradients['dW'+str(L)] = dW
    gradients['db'+str(L)] = db

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev, dW, db = linear_activation_backward(gradients['dA'+str(l+1)], current_cache, "relu")
        gradients['dA'+str(l)] = dA_prev
        gradients['dW'+str(l+1)] = dW
        gradients['db'+str(l+1)] = db
    
    return gradients


In [74]:
def update_parameters(parameters, gradients, learning_rate):
    L = len(parameters) // 2
    parameters = parameters.copy()
    for l in range(L):
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate * gradients['dW'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate * gradients['db'+str(l+1)]
    return parameters

In [77]:
def DNN_model(X, Y, layer_dims, learning_rate, num_iterations, print_cost):
    
    np.random.seed(2)
    costs = []
    parameters = initialize_parameters(layer_dims)
    for i in range(0, num_iterations):
        AL, caches = forward_propagation(X, parameters)
        cost = cost_function(AL, Y)
        gradients = backward_propagation(AL, Y, caches)
        parameters = update_parameters(parameters, gradients, learning_rate)
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print (f"Cost after iteration {i}: {cost}")
        if print_cost and i % 100 == 0:
            costs.append(cost)
    
    return parameters, costs

In [82]:
parameters, costs = DNN_model(train_set_x, train_set_y, LAYER_DIMS, learning_rate=0.03, num_iterations = 2500, print_cost = True)

Cost after iteration 0: 0.6932318084914451
Cost after iteration 100: 0.6928822520427091
Cost after iteration 200: 0.6915789541813728
Cost after iteration 300: 0.6861204932061763
Cost after iteration 400: 0.6688406144822115
Cost after iteration 500: 0.6452096357449842
Cost after iteration 600: 0.6392138436946139
Cost after iteration 700: 0.6240687515294746
Cost after iteration 800: 0.6170524172878054
Cost after iteration 900: 0.6100606055547391
Cost after iteration 1000: 0.5963982479781317
Cost after iteration 1100: 0.5889446837268478
Cost after iteration 1200: 0.5808302959841678
Cost after iteration 1300: 0.5711525928757509
Cost after iteration 1400: 0.5680644440647212
Cost after iteration 1500: 0.5275917785009744
Cost after iteration 1600: 0.5392010811727852
Cost after iteration 1700: 0.5418716605731323
Cost after iteration 1800: 0.4837432797888942
Cost after iteration 1900: 0.5480520361339802
Cost after iteration 2000: 0.5194555835940085
Cost after iteration 2100: 0.4567357805975715
